# ✅ GeoTIFF Consistency Checker (CRS / Resolution / Alignment)

This notebook helps you **verify** that a folder of GeoTIFFs share the same:
- Coordinate Reference System (CRS)
- Pixel resolution (x/y)
- Pixel alignment (affine transform grid)
- Band count, dtype (optional checks)

It will output:
- A **table** of per-file metadata
- A **summary** of unique CRS/resolution/alignment combos
- A CSV report (`tif_report.csv`)
- (Optional) Commands / code cells to **reproject** mismatched tiles to a common CRS

> Tip: Run the cells top to bottom. If you see mismatches, use the optional reproject section.


In [ ]:
# If needed, install dependencies (uncomment as appropriate)
# %pip install rasterio geopandas shapely pandas numpy tqdm


In [1]:
from pathlib import Path

# === Configure your input directory here ===
INPUT_DIR = Path("/Users/jayceebao/Desktop/tifs")  # <-- CHANGE THIS to your folder containing .tif files
OUTPUT_DIR = Path("/Users/jayceebao/Desktop/tif_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Input directory:", INPUT_DIR.resolve())
print("Output directory:", OUTPUT_DIR.resolve())


Input directory: /Users/jayceebao/Desktop/tifs
Output directory: /Users/jayceebao/Desktop/tif_outputs


In [3]:
import os
import math
from glob import glob
import pandas as pd
import numpy as np
import rasterio
from rasterio.coords import BoundingBox
from rasterio.errors import RasterioIOError
from tqdm import tqdm

def transform_key(transform):
    """
    Build a concise key for alignment comparison:
    (origin_x mod pixel_x, origin_y mod pixel_y, pixel_x, pixel_y, rotation terms)
    """
    a = transform.a      # pixel width
    e = transform.e      # pixel height (negative in north-up)
    b = transform.b      # rotation term
    d = transform.d      # rotation term
    xoff = transform.c   # origin x
    yoff = transform.f   # origin y

    # Alignment residues modulo pixel size (robust to large offsets)
    def mod_residue(offset, step):
        if step == 0: 
            return 0.0
        r = offset % abs(step)
        # snap tiny numeric noise
        return float(np.round(r, 9))

    return (
        float(np.round(mod_residue(xoff, a), 9)),
        float(np.round(mod_residue(yoff, e), 9)),
        float(np.round(a, 9)),
        float(np.round(e, 9)),
        float(np.round(b, 9)),
        float(np.round(d, 9)),
    )

tif_paths = sorted([p for p in INPUT_DIR.rglob("*.tif")])
print(f"Found {len(tif_paths)} GeoTIFFs")

rows = []
for p in tqdm(tif_paths):
    try:
        with rasterio.open(p) as src:
            crs_obj = src.crs
            crs_epsg = crs_obj.to_epsg() if crs_obj else None
            crs_str = crs_obj.to_string() if crs_obj else None
            res_x, res_y = src.res
            tf = src.transform
            tf_key = transform_key(tf)
            bounds: BoundingBox = src.bounds
            nodata = src.nodata
            count = src.count
            dtype = src.dtypes[0] if count > 0 else None
            width, height = src.width, src.height

            rows.append({
                "path": str(p),
                "name": p.name,
                "crs_epsg": crs_epsg,
                "crs_string": crs_str,
                "res_x": float(np.round(res_x, 9)),
                "res_y": float(np.round(res_y, 9)),
                "align_key": tf_key,  # for alignment grouping
                "transform": tuple(map(float, (tf.a, tf.b, tf.c, tf.d, tf.e, tf.f))),
                "bounds": (float(bounds.left), float(bounds.bottom), float(bounds.right), float(bounds.top)),
                "width": width,
                "height": height,
                "bands": count,
                "dtype": dtype,
                "nodata": nodata,
            })
    except RasterioIOError as e:
        rows.append({
            "path": str(p),
            "name": p.name,
            "crs_epsg": None,
            "crs_string": None,
            "res_x": None,
            "res_y": None,
            "align_key": None,
            "transform": None,
            "bounds": None,
            "width": None,
            "height": None,
            "bands": None,
            "dtype": None,
            "nodata": None,
            "error": str(e),
        })

df = pd.DataFrame(rows).sort_values("name").reset_index(drop=True)
df


Found 818 GeoTIFFs


100%|██████████| 818/818 [01:20<00:00, 10.12it/s]


,path,name,crs_epsg,crs_string,res_x,res_y,align_key,transform,bounds,width,height,bands,dtype,nodata
0,/Users/jayceebao/Desktop/tifs/patch_1375803_L_...,patch_1375803_L_2165_19.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 782100.0, 0.0, -10.0, 6711220.0)","(782100.0, 6706100.0, 787220.0, 6711220.0)",512,512,3,uint8,None
1,/Users/jayceebao/Desktop/tifs/patch_1375804_S_...,patch_1375804_S_2412_377.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 784570.0, 0.0, -10.0, 6707640.0)","(784570.0, 6705080.0, 787130.0, 6707640.0)",256,256,3,uint8,None
2,/Users/jayceebao/Desktop/tifs/patch_1375805_S_...,patch_1375805_S_2262_347.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 783070.0, 0.0, -10.0, 6707940.0)","(783070.0, 6705380.0, 785630.0, 6707940.0)",256,256,3,uint8,None
3,/Users/jayceebao/Desktop/tifs/patch_1375806_L_...,patch_1375806_L_2469_207.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 785140.0, 0.0, -10.0, 6709340.0)","(785140.0, 6704220.0, 790260.0, 6709340.0)",512,512,3,uint8,None
4,/Users/jayceebao/Desktop/tifs/patch_1375807_S_...,patch_1375807_S_2392_0.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 784370.0, 0.0, -10.0, 6711410.0)","(784370.0, 6708850.0, 786930.0, 6711410.0)",256,256,3,uint8,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,/Users/jayceebao/Desktop/tifs/patch_69420_S_64...,patch_69420_S_649_380.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 766940.0, 0.0, -10.0, 6707610.0)","(766940.0, 6705050.0, 769500.0, 6707610.0)",256,256,3,uint8,None
814,/Users/jayceebao/Desktop/tifs/patch_69421_S_83...,patch_69421_S_836_421.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 768810.0, 0.0, -10.0, 6707200.0)","(768810.0, 6704640.0, 771370.0, 6707200.0)",256,256,3,uint8,None
815,/Users/jayceebao/Desktop/tifs/patch_70438_L_14...,patch_70438_L_1472_272.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 775170.0, 0.0, -10.0, 6708690.0)","(775170.0, 6703570.0, 780290.0, 6708690.0)",512,512,3,uint8,None
816,/Users/jayceebao/Desktop/tifs/patch_70439_L_14...,patch_70439_L_1474_355.tif,None,"PROJCS[""WGS 84 / UTM zone 55S"",GEOGCS[""WGS 84""...",10.0,10.0,"(0.0, 0.0, 10.0, -10.0, 0.0, 0.0)","(10.0, 0.0, 775190.0, 0.0, -10.0, 6707860.0)","(775190.0, 6702740.0, 780310.0, 6707860.0)",512,512,3,uint8,None


In [4]:
from IPython.display import display
display(df)
report_csv = OUTPUT_DIR / "tif_report.csv"
df.to_csv(report_csv, index=False)
display_dataframe_to_user("GeoTIFF Metadata Report", df)
print("Saved CSV to:", report_csv.resolve())


ModuleNotFoundError: No module named 'caas_jupyter_tools'

In [ ]:
def summarize_unique(series):
    vals = series.dropna().unique().tolist()
    return len(vals), vals[:10]  # preview up to 10

summary = {
    "n_files": len(df),
    "unique_crs_epsg": summarize_unique(df["crs_epsg"]),
    "unique_crs_string": summarize_unique(df["crs_string"]),
    "unique_resolution_pairs": summarize_unique(df[["res_x","res_y"]].astype(str).agg(tuple, axis=1)),
    "unique_alignment_keys": summarize_unique(df["align_key"].astype(str)),
    "unique_dtype": summarize_unique(df["dtype"]),
    "unique_bands": summarize_unique(df["bands"]),
}

print("=== SUMMARY ===")
for k, v in summary.items():
    print(f"{k}: {v[0]} unique")
    if isinstance(v[1], list):
        print("  sample:", v[1])

# Identify groups by (crs_epsg, res_x, res_y, align_key)
df["res_pair"] = list(zip(df["res_x"], df["res_y"]))
group_cols = ["crs_epsg", "res_pair", "align_key"]
grouped = df.groupby(group_cols, dropna=False)

print("\n=== GROUPS (by CRS / resolution / alignment) ===")
for key, g in grouped:
    print(f"Group {key}: {len(g)} files")

# Flag inconsistencies
all_same_crs = summary["unique_crs_epsg"][0] == 1 or summary["unique_crs_string"][0] == 1
all_same_res = summary["unique_resolution_pairs"][0] == 1
all_same_align = summary["unique_alignment_keys"][0] == 1

print("\n=== CONSISTENCY CHECK ===")
print("Same CRS across all tiles:     ", "✅" if all_same_crs else "❌")
print("Same resolution across tiles:  ", "✅" if all_same_res else "❌")
print("Same alignment across tiles:   ", "✅" if all_same_align else "❌")

# Save a per‑group listing to disk for quick inspection
with open(OUTPUT_DIR / "groups.txt", "w") as f:
    for key, g in grouped:
        f.write(f"Group {key}: {len(g)} files\n")
        for _, row in g.iterrows():
            f.write(f"  - {row['name']}\n")
        f.write("\n")
print("Wrote group listing to:", (OUTPUT_DIR / "groups.txt").resolve())


In [ ]:
# Choose a target CRS as the most frequent crs_epsg among files (fallback to first non-null crs_string)
if df["crs_epsg"].notna().any():
    target_crs = int(df["crs_epsg"].mode().iloc[0])
    target_crs_str = f"EPSG:{target_crs}"
else:
    # Fallback to any string if EPSG missing
    target_crs_str = df["crs_string"].dropna().mode().iloc[0] if df["crs_string"].notna().any() else None

print("Auto-selected target CRS:", target_crs_str)


In [ ]:
from pathlib import Path

mismatch = df[(df["crs_string"].notna()) & (df["crs_string"] != target_crs_str)]
suggestions_path = OUTPUT_DIR / "reproject_suggestions.sh"

with open(suggestions_path, "w") as f:
    f.write("#!/usr/bin/env bash\n\n")
    f.write("# Suggested gdalwarp commands to reproject tiles to a common CRS\n")
    f.write(f"# Target CRS: {target_crs_str}\n\n")
    outdir = OUTPUT_DIR / "reprojected"
    f.write(f"mkdir -p {outdir.as_posix()}\n\n")
    for _, row in mismatch.iterrows():
        src = Path(row["path"])
        dst = outdir / src.name
        f.write(f'gdalwarp -t_srs "{target_crs_str}" -r near -co COMPRESS=LZW "{src.as_posix()}" "{dst.as_posix()}"\n')
print("Wrote reproject suggestions to:", suggestions_path.resolve())


## (Optional) Reproject with Rasterio (Python)

If you prefer Python instead of GDAL CLI, run the cell below.  
It will create `OUTPUT_DIR/reprojected` and write reprojected copies for files that don't match the target CRS.


In [ ]:
# WARNING: This can be slow for many large tiles. Use GDAL for speed when possible.
from rasterio.warp import calculate_default_transform, reproject, Resampling

OUT_REPROJ = OUTPUT_DIR / "reprojected"
OUT_REPROJ.mkdir(parents=True, exist_ok=True)

if target_crs_str is None:
    raise ValueError("No target CRS could be determined. Please set `target_crs_str` manually.")

target_crs_rio = rasterio.crs.CRS.from_string(target_crs_str)

for _, row in df.iterrows():
    src_path = Path(row["path"])
    if pd.isna(row["crs_string"]) or row["crs_string"] == target_crs_str:
        continue  # already aligned CRS or missing CRS

    with rasterio.open(src_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs_rio, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            "crs": target_crs_rio,
            "transform": transform,
            "width": width,
            "height": height,
            "compress": "lzw",
        })

        dst_path = OUT_REPROJ / src_path.name
        with rasterio.open(dst_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs_rio,
                    resampling=Resampling.nearest,
                )
print("Reprojection (rasterio) complete. Output folder:", OUT_REPROJ.resolve())


## (Optional) Enforce a Common Resolution

If some tiles have different pixel sizes, you can set a target resolution and resample.  
**Note:** This step is independent from CRS reprojection; you can combine both by using `gdalwarp` with `-tr xres yres`.


In [ ]:
# Example: set a standard resolution (in target CRS units). Uncomment and adjust to use.
# target_res_x, target_res_y = 10.0, 10.0  # e.g., 10m Sentinel‑2 style
# suggestions_resample = OUTPUT_DIR / "resample_suggestions.sh"
# with open(suggestions_resample, "w") as f:
#     f.write("#!/usr/bin/env bash\n\n")
#     f.write("# Suggested gdalwarp commands to resample to a common resolution\n")
#     f.write(f"# Target resolution: {target_res_x} x {target_res_y}\n\n")
#     outdir = OUTPUT_DIR / "resampled"
#     f.write(f"mkdir -p {outdir.as_posix()}\n\n")
#     for _, row in df.iterrows():
#         src = Path(row['path'])
#         dst = outdir / src.name
#         f.write(f'gdalwarp -tr {target_res_x} {target_res_y} -r bilinear -co COMPRESS=LZW "{src.as_posix()}" "{dst.as_posix()}"\n')
# print("Wrote resample suggestions to:", suggestions_resample.resolve())
